In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import main as m
import regex as re

In [2]:
# investigator 
# ablilities with embedid values not capturing correctly
# Complex deck reqs not being captured correctly
# some must includes not being captured correctly

In [3]:
invest, assets, events, skills = m.get_urls_by_type()


Getting investigator URL's...
Getting asset URL's...
Getting event URL's...
Getting skill URL's...


In [4]:
##########################################Get soup request#########################################################
def get_soup(url):
    '''Child of get_skills_df
       Takes in a url for a card
       Returns html request result parsed using beautiful soup'''

    # create request and soup objects
    html = requests.get(url)

    soup = BeautifulSoup(html.content, 'html.parser')

    # locate urls on page and return
    return soup.find(id='list')

In [5]:
def clean_ability_string(ability):
    
    # replace icon html with matching word in all caps
    icon_types = [
                  'reaction',
                  'wild', 
                  'willpower', 
                  'combat', 
                  'agility', 
                  'intellect', 
                  'wild',
                  'curse', 
                  'bless',
                  'rogue',
                  'survivor',
                  'seeker',
                  'guardian',
                  'mystic',
                  'neutral',
                  'skull',
                  'tablet',
                  'cultist',
                  'elder sign']
    
    for icon in icon_types:
    
        ability = ability.replace(f'<span class="icon-{icon}" title="{icon.capitalize()}"></span>', 
                                  f'{icon.upper()}')
        
        ability = ability.replace(f'<div class="card-text border-{icon}">\n<p>', '')

    ability = ability.replace(f'<span class="icon-wild" title="Any Skill"></span>', 'WILD')

    ability = ability.replace(f'<span class="icon-elder_sign" title="Elder Sign"></span>', 'ELDER_SIGN')

    ability = ability.replace(f'<span class="icon-elder_sign" title="Elder Thing"></span>', 'ELDER_THING')

    # delete extraneous html
    dirt = [
            '</p>\n</div>',
            '</p>',
            '<p>',
            '<b>',
            '</b>',
            '<br/',
            '<1>',
            '<i>',
            '</i>',
            '><span>',]
    
    for item in dirt:
        
        ability = ability.replace(item,'')

    return ability

In [6]:
def get_stat_line(results):
    
    stat_line = results.find('div', class_='card-props').text.replace('\n', '').replace('\t', '')

    stat_list = [int(char) for char in str(stat_line) if char.isnumeric() == True]

    return stat_list[0], stat_list[1], stat_list[2], stat_list[3]


def get_stam_line(results):
    
    stat_line = results.find('div', class_='card-props').find_next().text.replace('\n', '').replace('\t', '')

    stat_list = [int(char) for char in str(stat_line) if char.isnumeric() == True]

    return stat_list[0], stat_list[1]


def get_deck_reqs(results, faction):

    deck = results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text

    pattern = r'(?:\b(?:Deck Size|Deckbuilding Options|Deckbuilding Requirements|Additional Setup):|\b\s*)([^:\n]+)'

    deck_split = re.findall(pattern, str(deck))

    deck_size = re.search('[0-9]{2}', deck_split[0]).group()

    deck_options = deck_split[1].replace('Deckbuilding Requirements (do not count toward deck size)', '').replace('(', '').replace(')', '').replace('  ', ' ')

    must_include = deck_split[2].replace('.Additional Setup', '')

    try:

        setup = deck_split[3]

    except:

        setup = NONE

    return deck_size, deck_options, must_include, setup

In [7]:
def get_invest_traits(results):
    '''Scrapes and returns investigator card data'''

    title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')

    faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

    tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

    traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')

    willpower, intellect, combat, agility = get_stat_line(results)

    health, sanity = get_stam_line(results)
   
    ability = results.find('div', class_=f'card-text border-{faction.lower()}').text.replace('\n', '').replace('\t', '')

    artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

    expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '') 

    try:
        
        flavor = results.find_all('div', class_='card-flavor small')[1].text.replace('\n', '').replace('\t', '')

    except:
        
        flavor = ''
        
    deck_size, deck_options, must_include, setup = get_deck_reqs(results, faction)
    
    return [title, 
            faction, 
            tipe, 
            traits, 
            willpower, 
            intellect, 
            combat, 
            agility, 
            health, 
            sanity,
            ability, 
            artist,
            expansion,
            flavor,
            deck_size,
            deck_options,
            must_include,
            setup]

In [8]:
# make empty df

def get_invest_df():

    investigator_traits = { 'title':[],
                            'faction':[],
                            'type':[],
                            'traits':[],                      
                            'willpower':[],
                            'intellect':[],
                            'combat':[],
                            'agility':[],                       
                            'health':[],
                            'sanity':[],                     
                            'ability':[],
                            'artist':[],
                            'expansion':[],
                            'flavor':[],
                            'deck_size':[],
                            'deck_options':[],
                            'must_include':[],
                            'setup':[]}

    # for each url get player card info from that page and add each element to skill_traits
    for url in invest:

        # make html request to arkham db and parse using BS
        results = get_soup(url)

        # get list of card elements card elements
        trait_list = get_invest_traits(results)

        print(f'Getting investigator card {trait_list[0]}...')

        # itterate through card element titles and add each to a dictionary
        for i, key in enumerate(investigator_traits):

            investigator_traits[key].append(trait_list[i])

    print("Making dataframe...")

    return pd.DataFrame(investigator_traits)

In [9]:
get_invest_df()

NameError: name 'NONE' is not defined

In [10]:
# [title, test_icons, traits, willpower, intellect, combat, agility, health, sanity, faction, ability, tipe, flavor, artist, expansion]

In [11]:
# Investigator Traits

results = get_soup('https://arkhamdb.com/card/06003')

title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')

faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')

willpower, intellect, combat, agility = get_stat_line(results)

health, sanity = get_stam_line(results)

ability = results.find('div', class_=f'card-text border-{faction.lower()}').text.replace('\n', '').replace('\t', '')

artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '') 

flavor = results.find_all('div', class_='card-flavor small')[1].text.replace('\n', '').replace('\t', '')

deck_size, deck_options, must_include = get_deck_reqs(results)

TypeError: get_deck_reqs() missing 1 required positional argument: 'faction'

In [58]:
results = get_soup('https://arkhamdb.com/card/06003')

faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text.replace('\n', '').replace('\t', '')

"Deck Size: 30.Secondary Class Choice: At deck creation, choose Guardian (), Seeker (), or Survivor ().Deckbuilding Options: Rogue cards () level 0-5, Neutral cards level 0-5, up to 10 level 0-1 events and/or skills of your chosen secondary class.Deckbuilding Requirements (do not count toward deck size): Bounty Contracts, 2 copies of Tony's .38 Long Colt, Tony's Quarry, 1 random basic weakness."

In [ ]:
results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text



In [12]:
def get_deck_reqs(results, faction):
    
    deck = results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text

    deck_split = re.split(':', deck)

    deck_size = re.search('[0-9]{2}', deck_split[1]).group()
    
    deck_options = re.search('^(.*?\.)', deck_split[2]).group()[1:-1].replace('(', '').replace(')', '').replace('  ', ' ')

    must_include = re.search('^(.*?\.)', deck_split[3]).group()[1:-1].replace('(', '').replace(')', '').replace('  ', ' ')
    
    return deck_size, deck_options, must_include

In [13]:


def get_deck_reqs(results, faction):

    deck = results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text

    pattern = r'(?:\b(?:Deck Size|Deckbuilding Options|Deckbuilding Requirements|Additional Setup):|\b\s*)([^:\n]+)'

    deck_split = re.findall(pattern, str(deck))

    deck_size = re.search('[0-9]{2}', deck_split[0]).group()

    deck_options = deck_split[1].replace('Deckbuilding Requirements (do not count toward deck size)', '').replace('(', '').replace(')', '').replace('  ', ' ')

    must_include = deck_split[2].replace('.Additional Setup', '')

    try:

        setup = deck_split[3]

    except:

        setup = NONE

    return deck_size, deck_options, must_include, setup

In [52]:
results = get_soup(invest[26])

faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

deck = results.find('div', class_=f'card-text border-{faction.lower()}').text

pattern = r'(?:\b(?:Deck Size):|\b\s*)([^:\n]+)'

deck_split = re.findall(pattern, deck)

deck_split

['When an investigator at your location would reveal any number of chaos tokens',
 'Reveal 2 additional tokens. Of the revealed tokens, choose and cancel 2 non- tokens, or 1  token. (Limit once per round.) effect',
 '1. If this effect is canceled or ignored, draw 1 card.']

In [15]:
get_soup(assets[20])

<div id="list">
<div class="row">
<div class="col-sm-12">
<ul class="pager">
<li class="previous">
<a class="no-popup" href="/card/05039">← Five of Pentacles</a> </li>
<li>
<a href="/set/tcu">The Circle Undone</a>
</li>
<li class="next">
<a class="no-popup" href="/card/05041">The 13th Vision →</a> </li>
</ul>
</div>
</div>
<div class="row">
<div class="col-md-9">
<div class="row">
<div class="col-sm-7">
<div>
<div class="panel panel-default border-neutral">
<div class="panel-heading border-neutral bg-neutral">
<h3 class="panel-title">
<a class="card-name card-tip" data-code="05040" href="https://arkhamdb.com/card/05040">Ace of Rods
</a>
<div class="card-subname small">The Fateful Step</div> </h3>
</div>
<div class="panel-body card-content">
<div class="card-faction">
<span class="icon-neutral fg-neutral"></span>
<span class="card-faction">Neutral</span>
</div>
<div class="card-info">
<span class="card-type">Asset. </span>
<p class="card-traits">Tarot.</p>
<div class="card-props">
		Cos

In [53]:
# Input string
input_string = '''Deck Size: 30 Secondary Class Choice: At deck creation, choose Guardian (), Seeker (), 
                or Survivor (). Deckbuilding Options: Rogue cards () level 0-5, Neutral cards level 0-5, 
                up to 10 level 0-1 events and/or skills of your chosen secondary class. Deckbuilding Requirements 
                (do not count toward deck size): Bounty Contracts, 2 copies of Tony's .38 Long Colt, Tony's Quarry,
                1 random basic weakness.'''

# Regex patterns
deck_size_pattern = re.compile(r"Deck Size:\s*(\d+)")
secondary_class_pattern = re.compile(r"Secondary Class Choice:\s*(Guardian|Seeker|Survivor)")
deck_options_pattern = re.compile(r"Deckbuilding Options:\s*([^\.]+)\.")
deck_requirements_pattern = re.compile(r"Deckbuilding Requirements:\s*([^\.]+)\.")

# Extract information
deck_size_match = deck_size_pattern.search(input_string)
secondary_class_match = secondary_class_pattern.search(input_string)
deck_options_match = deck_options_pattern.search(input_string)
deck_requirements_match = deck_requirements_pattern.search(input_string)

# Extracted variables or set to None if not present
deck_size = deck_size_match.group(1) if deck_size_match else None
secondary_class_choice = secondary_class_match.group(1) if secondary_class_match else None
deck_options = deck_options_match.group(1).strip() if deck_options_match else None
deck_requirements = deck_requirements_match.group(1).strip() if deck_requirements_match else None

# Print the results
print("Deck Size:", deck_size)
print("Secondary Class Choice:", secondary_class_choice)
print("Deckbuilding Options:", deck_options)
print("Deckbuilding Requirements:", deck_requirements)

Deck Size: 30
Secondary Class Choice: None
Deckbuilding Options: Rogue cards () level 0-5, Neutral cards level 0-5, 
                up to 10 level 0-1 events and/or skills of your chosen secondary class
Deckbuilding Requirements: None


In [16]:
title

'Diana Stanley'

In [17]:
faction

'Mystic'

In [18]:
tipe

'Investigator'

In [19]:
traits

'Cultist. Silver Twilight.'

In [20]:
willpower

1

In [21]:
intellect

3

In [22]:
combat

3

In [23]:
agility

3

In [24]:
health

7

In [25]:
sanity

7

In [26]:
ability

'Diana Stanley gets +1  for each card beneath her. After a card you own cancels or ignores a card effect or game effect, if there are fewer than 5 cards beneath Diana Stanley: Place that card facedown beneath her. Draw 1 card and gain 1 resource. (Limit once per phase.) effect: +2. You may choose a card beneath Diana Stanley and add it to your hand.'

In [27]:
artist

'Magali Villeneuve'

In [28]:
expansion

'The Circle Undone #4'

In [29]:
deck_size

NameError: name 'deck_size' is not defined

In [30]:
flavor

"As a new business owner in Arkham, Diana Stanley was proud to be inducted into the prestigious Silver Twilight Lodge. Her joy has since turned to dread. Her admittance into the organization was fantastic for business, but the weekly meetings have grown more and more disturbing. After witnessing strange rituals and terrible secrets, Diana has become suspicious of the Lodge's true nature. Now she is convinced that there are terrible forces converging on Arkham, and that the Lodge is somehow involved. Diana has resolved to take it down from the inside, regardless of the cost."

In [31]:
deck_size

NameError: name 'deck_size' is not defined

In [32]:
deck_options

NameError: name 'deck_options' is not defined

In [33]:
must_include 

NameError: name 'must_include' is not defined